In [ ]:
# The point of this notebook is to combine and collect all the
# necessary information for the individual players

PLAYER_URL = "https://github.com/vaastav/Fantasy-Premier-League/blob/master/data/{}/cleaned_players.csv"


In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import re
import enum
from enum import Enum
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get the functions from the util file
!cp '/content/drive/MyDrive/FPL-Project/FPL-Utils.ipynb' /content/
%run /content/FPL-Utils.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
years = ["2020-21", "2021-22", "2022-23"]
previous_years = ["2019-20", "2020-21", "2021-22"]
gws = [f"gw{i}" for i in range(1, 39)]

list_dfs = []

# @TODO: figure out the encoding issue using the UTIL file
for i, year in enumerate(years):
    print(f"Collecting data for year:{year}")

    previous_stats_url = f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{previous_years[i]}/cleaned_players.csv"
    encoding_prev_stats = detect_encoding(previous_stats_url)

    # get previous_seasons_data
    player_prev_stats = pd.read_csv(
        f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{previous_years[i]}/cleaned_players.csv",
        encoding=encoding_prev_stats,
    )
    player_prev_stats["name"] = (
        player_prev_stats["first_name"] + " " + player_prev_stats["second_name"]
    )
    player_prev_stats.drop(["first_name", "second_name"], axis=1, inplace=True)
    player_prev_stats.columns = player_prev_stats.columns + "_previous_season"

    teams_year_url = f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{year}/teams.csv"
    encoding_teams = detect_encoding(teams_year_url)

    # get opponent_team
    teams = pd.read_csv(
        f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{year}/teams.csv",
        encoding=encoding_teams,
    )[["id", "name"]]
    teams.columns = ["opponent_team", "opponent"]

    # opponents position last season
    teams["opponent_last_season_position"] = teams["opponent"].apply(
        get_teams_previous_position(year)
    )

    for gameweek in range(1, 39):
        print(f"Collecting data for gameweek:{gameweek}")

        gameweek_url = f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{year}/gws/gw{gameweek}.csv"
        encoding_gameweek = detect_encoding(gameweek_url)
        df = pd.read_csv(
            f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{year}/gws/gw{gameweek}.csv",
            encoding=encoding_gameweek,
        )

        # teams position last season
        df["last_season_position"] = df["team"].apply(get_teams_previous_position(year))

        # calculate percentage value to team
        df["percent_value"] = df["name"].apply(lambda name: calculate_point_ratio_against_team(name, df))

        # chek if the result was a win or not
        df['match_status'] = df.apply(match_status, axis=1)

        # merge previous_season_data
        df = pd.merge(
            df, player_prev_stats, left_on="name", right_on="name_previous_season", how="left"
        )
        df["season"] = year
        df.drop("name_previous_season", axis=1, inplace=True)
        df["GW"] = gameweek

        # merge opponent team
        df = pd.merge(df, teams, on="opponent_team", how="left")
        df = df.fillna(0)
        list_dfs.append(df)

all_dfs = pd.concat(list_dfs)
all_dfs.to_csv("/content/drive/MyDrive/FPL-Project/cleaned_previous_seasons_gameweeks.csv", index=False)
print("All Done, Saved the file!")

<ipython-input-4-8d31d0fda379>:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_dfs = pd.concat(list_dfs)


All Done, Saved the file!


In [ ]:
def get_week():
    df = pd.read_csv(
        f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/gws/gw1.csv",
        encoding="latin-1",
    )

            # teams position last season
    df["last_season_position"] = df["team"].apply(get_teams_previous_position("2023-24"))

    # calculate percentage value to team
    df["percent_value"] = df["name"].apply(lambda name: calculate_point_ratio_against_team(name, df))

    # chek if the result was a win or not
    df['match_status'] = df.apply(match_status, axis=1)

    df["season"] = "2023-24"
    df["GW"] = "1"

    df.to_csv("/content/drive/MyDrive/FPL-Project/clean_gameweeks_predict.csv", index=False)
    print("All Done, Saved the file!")

get_week()

All Done, Saved the file!
